In [ ]:
# Install Dependencies
!pip install -q "demucs" "fastapi" "uvicorn" "pyngrok" "python-multipart" "nest-asyncio"

import os
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import FileResponse
from demucs.apply import apply_model
from demucs.pretrained import get_model
from demucs.audio import save_audio
import torch
import torchaudio
import uvicorn
from pyngrok import ngrok
import nest_asyncio
import shutil

In [ ]:
# Initialize App
app = FastAPI()
os.makedirs("input", exist_ok=True)
os.makedirs("separated", exist_ok=True)

# Load Model (GPU check included)
print("Loading Demucs Model...")
model = get_model('htdemucs_ft')
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(f"Model Loaded on {device}")

@app.post("/separate")
async def separate_audio(file: UploadFile = File(...)):
    file_path = f"input/{file.filename}"
    with open(file_path, "wb") as buffer:
        shutil.copyfileobj(file.file, buffer)
    
    # Load and Separate
    wav, sr = torchaudio.load(file_path)
    wav = wav.to(device)
    sources = apply_model(model, wav[None], shifts=1, split=True, overlap=0.1)[0]
    
    # Save & Zip
    output_dir = f"separated/{file.filename.split('.')[0]}"
    os.makedirs(output_dir, exist_ok=True)
    for source, name in zip(sources, model.sources):
        save_audio(source, f"{output_dir}/{name}.mp3", samplerate=sr)
    
    shutil.make_archive(output_dir, 'zip', output_dir)
    return FileResponse(f"{output_dir}.zip")

@app.get("/")
def home():
    return {"status": "SonicSplit Server Running"}

# Start Server
# !ngrok config add-authtoken <YOUR_TOKEN_HERE>  <-- Optional: Agar token daalna ho
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)